In [2]:
import geopandas as gp
%matplotlib inline
import pandas as pd

In [3]:
import folium

# Colombia

In [95]:
shp_state = gp.read_file('data/colombia/ADMINISTRATIVO/MGN_DPTO_POLITICO.shp')
col_state = ['DPTO_CCDGO','DPTO_CNMBR', 'geometry']
shp_munic = gp.read_file('data/colombia/ADMINISTRATIVO/MGN_MPIO_POLITICO.shp')
col_munic = ['DPTO_CCDGO','MPIO_CCDGO', 'MPIO_CNMBR', 'DPTO_CNMBR', 'geometry']
shp_state.shape, shp_munic.shape

((33, 9), (1122, 11))

In [98]:
# CLAS_CCDGO: 1-Municipal Headboard 2-populated center 3-scattered rural area 
shp_urb_censal = gp.read_file('data/colombia/MGN/MGN_URB_AREA_CENSAL.shp')
print(shp_urb_censal.shape)
shp_urb_censal=shp_urb_censal.merge(shp_munic[col_munic[:-1]])
shp_urb_censal.head()

(7627, 14)


,OBJECTID,DPTO_CCDGO,MPIO_CCDGO,CLAS_CCDGO,SETR_CCDGO,SECR_CCDGO,CPOB_CCDGO,CPOB_CNMBR,CPOB_CCNCT,CPOB_NAREA,CPOB_NALT,Shape_Leng,Shape_Area,geometry,MPIO_CNMBR,DPTO_CNMBR
0,1,05,05001,2,000,00,05001004,SANTA ELENA,05001200000004,0.338461,2540.0,0.034407,0.000028,POLYGON ((-75.49783904499998 6.207774365000034...,MEDELLÍN,ANTIOQUIA
1,2,05,05001,2,000,00,05001014,BARRO BLANCO,05001200000014,0.534495,2531.0,0.044447,0.000044,POLYGON ((-75.47379840599996 6.239590113000077...,MEDELLÍN,ANTIOQUIA
2,3,05,05001,2,000,00,05001015,EL CERRO,05001200000015,0.414935,2668.0,0.042879,0.000034,(POLYGON ((-75.49121943999995 6.20501584900006...,MEDELLÍN,ANTIOQUIA
3,4,05,05001,2,000,00,05001018,EL PLACER,05001200000018,0.965299,2598.0,0.056312,0.000079,"POLYGON ((-75.49276999399996 6.23050003000003,...",MEDELLÍN,ANTIOQUIA
4,5,05,05001,2,000,00,05001019,EL PLAN,05001200000019,3.252561,2700.0,0.093795,0.000266,POLYGON ((-75.50917742799999 6.195200505000059...,MEDELLÍN,ANTIOQUIA


In [135]:
# print('\n'.join(shp_urb_censal.CPOB_CNMBR.unique().tolist()))
# shp_urb_censal.CPOB_CNMBR.nunique()

In [107]:
shp_urb_sector = gp.read_file('data/colombia/MGN/MGN_URB_SECTOR.shp')
print(shp_urb_sector.shape)
shp_urb_sector.head()

(11683, 12)


,OBJECTID,DPTO_CCDGO,MPIO_CCDGO,CLAS_CCDGO,SETR_CCDGO,SECR_CCDGO,CPOB_CCDGO,SETU_CCDGO,SETU_CCNCT,Shape_Leng,Shape_Area,geometry
0,1,05,05001,2,003,04,05001004,0001,050012003040040001,0.034407,0.000028,POLYGON ((-75.49783904499998 6.207774365000034...
1,2,05,05001,2,003,02,05001014,0001,050012003020140001,0.044447,0.000044,POLYGON ((-75.47379840599996 6.239590113000077...
2,3,05,05001,2,003,04,05001015,0001,050012003040150001,0.042879,0.000034,(POLYGON ((-75.49121943999995 6.20501584900006...
3,4,05,05001,2,003,03,05001018,0001,050012003030180001,0.056312,0.000079,"POLYGON ((-75.49276999399996 6.23050003000003,..."
4,5,05,05001,2,003,04,05001019,0001,050012003040190001,0.093795,0.000266,POLYGON ((-75.51171155399999 6.193787105000069...


In [111]:
def geojson_per_row(gpdf, name, color='blue', tip_cols=None, some_map=None):

    feature_group = folium.FeatureGroup(name=name)
    for row in gpdf.itertuples():

        if row.geometry.boundary.type=='MultiLineString':
            lines = row.geometry.boundary
        else:
            lines= [row.geometry.boundary]
            
        for line in lines:
            print
            tip = '<br>'.join(['%s: %s' % (col, getattr(row, col)) for col in tip_cols]) if tip_cols is not None else name
            folium.Polygon(locations=[(lat,lon) for lon,lat in line.coords], color=color, fill_color=color, tooltip=tip, popup=tip).add_to(feature_group)
    if some_map is not None:
        feature_group.add_to(some_map)
    return feature_group

In [112]:
some_map = folium.Map(location=[4.149624, -72.795576], zoom_start=6)
fg_urb_censal = geojson_per_row(shp_urb_censal, 'urban censal', color='blue', 
                                tip_cols=['DPTO_CNMBR', 'MPIO_CNMBR', 'CPOB_CNMBR'], some_map=some_map)
fg_urb_sector = geojson_per_row(shp_urb_sector, 'urban sector', color='red', tip_cols=['SETU_CCNCT'], some_map=some_map)
folium.LayerControl().add_to(some_map)
some_map.save('col urb .html')

## build map for amazonas departo

In [47]:
def load_popup(path, m):
    shp = gp.read_file(path)
    geojson = folium.GeoJson(shp[['geometry']], tooltip=path)
    popup = folium.Popup(path)
    popup.add_to(geojson)
    geojson.add_to(m)


In [40]:
import glob

In [66]:
gp.read_file('data/colombia/91_AMAZONAS/URBANO/MGN_URB_MANZANA.shp')

,DPTO_CCDGO,MPIO_CCDGO,CLAS_CCDGO,SETR_CCDGO,SECR_CCDGO,CPOB_CCDGO,SETU_CCDGO,SECU_CCDGO,MANZ_CCDGO,MANZ_CCNCT,MANZ_CAG,Shape_Leng,Shape_Area,geometry
0,91,91001,1,000,00,91001000,0002,01,07,9100110000000000020107,962409,0.004879,1.087139e-06,POLYGON ((-69.94301754299994 -4.21455729099994...
1,91,91001,1,000,00,91001000,0002,01,08,9100110000000000020108,962410,0.004172,1.085916e-06,POLYGON ((-69.94331568399997 -4.21339935899993...
2,91,91001,1,000,00,91001000,0001,01,12,9100110000000000010112,962351,0.004531,1.272354e-06,POLYGON ((-69.94369970299994 -4.21108022699996...
3,91,91001,1,000,00,91001000,0001,01,07,9100110000000000010107,962346,0.005017,1.545320e-06,POLYGON ((-69.94446238899997 -4.21009204699993...
4,91,91001,1,000,00,91001000,0001,01,06,9100110000000000010106,962345,0.005703,2.040979e-06,POLYGON ((-69.94478917999999 -4.20926348899996...
5,91,91001,1,000,00,91001000,0001,01,04,9100110000000000010104,962343,0.005661,1.982000e-06,POLYGON ((-69.94507235999998 -4.20710072599996...
6,91,91001,1,000,00,91001000,0001,02,04,9100110000000000010204,962359,0.004119,1.061435e-06,POLYGON ((-69.94328599699998 -4.20822383199993...
7,91,91001,1,000,00,91001000,0001,02,06,9100110000000000010206,962361,0.004530,1.263894e-06,POLYGON ((-69.94275723599998 -4.21063147499995...
8,91,91001,1,000,00,91001000,0001,02,09,9100110000000000010209,962364,0.004310,1.157067e-06,POLYGON ((-69.94276569399995 -4.21071124699994...
9,91,91001,1,000,00,91001000,0002,01,10,9100110000000000020110,962412,0.004230,1.117555e-06,POLYGON ((-69.94248292099996 -4.21197233999993...


In [113]:
for fn in glob.glob('data/colombia/91_AMAZONAS/*/*.shp'):
    print("load_popup('%s', some_map)" % fn)
#     shp = gp.read_file(fn)
#     print('=======',fn)
#     print(list(shp.columns))

load_popup('data/colombia/91_AMAZONAS/ADMINISTRATIVO/MGN_DPTO_POLITICO.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/ADMINISTRATIVO/MGN_MPIO_POLITICO.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/COLOMBIA/COLOMBIA.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/MGN/MGN_RUR_SECCION.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/MGN/MGN_RUR_SECTOR.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/MGN/MGN_URB_AREA_CENSAL.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/MGN/MGN_URB_SECCION.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/MGN/MGN_URB_SECTOR.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_CULTU_LINEA.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_CULTU_PUNTO.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_CULTU_TEXTO.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_HIDRO_LINEA.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_HIDRO_PUNT

In [64]:
some_map = folium.Map(location=[-1.420758, -71.293515], zoom_start=7)
# load_popup('data/colombia/91_AMAZONAS/ADMINISTRATIVO/MGN_DPTO_POLITICO.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/ADMINISTRATIVO/MGN_MPIO_POLITICO.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/COLOMBIA/COLOMBIA.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/MGN/MGN_RUR_SECCION.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/MGN/MGN_RUR_SECTOR.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/MGN/MGN_URB_AREA_CENSAL.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/MGN/MGN_URB_SECCION.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/MGN/MGN_URB_SECTOR.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_CULTU_LINEA.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_CULTU_PUNTO.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_CULTU_TEXTO.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_HIDRO_LINEA.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_HIDRO_PUNTO.shp', some_map)
# load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_HIDRO_TEXTO.shp', some_map)
#         load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_HIPSO_LINEA.shp', some_map)
#         load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_HIPSO_PUNTO.shp', some_map)
#         load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_HIPSO_TEXTO.shp', some_map)
#         load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_OTRLI_LINEA.shp', some_map)
#         load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_OTRLI_PUNTO.shp', some_map)
#         load_popup('data/colombia/91_AMAZONAS/RURAL/CBD_RUR_OTRLI_TEXTO.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/URBANO/MGN_URB_LINEA.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/URBANO/MGN_URB_MANZANA.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/URBANO/MGN_URB_NOMENCLATURA_VIAL.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/URBANO/MGN_URB_TOPONIMIA.shp', some_map)
load_popup('data/colombia/91_AMAZONAS/VIAS/VIAS.shp', some_map)

folium.LayerControl().add_to(some_map)
some_map.save('AMAZONAS.html')